In [1]:
# 모듈 import

from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets import (build_dataloader, build_dataset,
                            replace_ImageToTensor)
from mmdet.utils import get_device

import argparse
import wandb


/opt/conda/lib/python3.10/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


In [4]:
from itertools import product

# 설정할 수 있는 모든 조합을 생성합니다.
combinations = list(product([0.9, 0.5], [0.5, 0.0], [0.3, 0.5], [0.1, 0.7]))

# 각 조합에 대해 모델을 훈련합니다.
for pos_iou_thr, neg_iou_thr, min_pos_iou, score_thr in combinations:
    # 클래스 정의
    classes = ("General trash", "Paper", "Paper pack", "Metal", "Glass", 
               "Plastic", "Styrofoam", "Plastic bag", "Battery", "Clothing")

    # 설정 파일 로드
    cfg = Config.fromfile('./configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py')
    root='../../../../../dataset/'

    # 데이터셋 설정 수정
    cfg.data.train.classes = classes
    cfg.data.train.img_prefix = root
    cfg.data.train.ann_file = root + 'train_80.json' 
    cfg.data.train.pipeline[2]['img_scale'] = (512,512) 

    cfg.data.val.classes = classes
    cfg.data.val.img_prefix = root
    cfg.data.val.ann_file = root + 'val_20.json' 
    cfg.data.val.pipeline[1]['img_scale'] = (512,512) 

    cfg.data.test.classes = classes
    cfg.data.test.img_prefix = root
    cfg.data.test.ann_file = root + 'test.json' 
    cfg.data.test.pipeline[1]['img_scale'] = (512,512) 
    cfg.data.samples_per_gpu = 2
    cfg.data.workers_per_gpu = 2

    cfg.seed = 20
    cfg.gpu_ids = [0]

    cfg.model.roi_head.bbox_head.num_classes = 10

    cfg.optimizer_config.grad_clip = dict(max_norm=35, norm_type=2)
    cfg.checkpoint_config = dict(max_keep_ckpts=3, interval=1)
    cfg.device = get_device()

    cfg.runner.max_epochs = 20
    # 모델 설정
    cfg.model.roi_head.bbox_head.num_classes = 10
    cfg.model.train_cfg.rpn.assigner.pos_iou_thr = pos_iou_thr
    cfg.model.train_cfg.rpn.assigner.neg_iou_thr = neg_iou_thr
    cfg.model.train_cfg.rpn.assigner.min_pos_iou = min_pos_iou
    cfg.model.train_cfg.rcnn.assigner.pos_iou_thr = pos_iou_thr
    cfg.model.train_cfg.rcnn.assigner.neg_iou_thr = neg_iou_thr
    cfg.model.train_cfg.rcnn.assigner.min_pos_iou = min_pos_iou
    cfg.model.test_cfg.rcnn.score_thr = score_thr

    # 모델 이름 생성
    model_name = f"faster_rcnn_fpn_pos_iou_thr{pos_iou_thr}_neg_iou_thr{neg_iou_thr}_min_pos_iou{min_pos_iou}_score_thr{score_thr}"
    cfg.work_dir = './work_dirs/'+ model_name

    # 로그 설정
    cfg.log_config.hooks = [
        dict(type='TextLoggerHook'),
        dict(type='MyMMDetWandbHook',
                init_kwargs={'project' : 'object-detection',
                            'name' : model_name},
                interval=10,
                log_checkpoint=True,
                log_checkpoint_metadata=True,
                num_eval_images=100
                )
                ]

    # 모델 훈련
    datasets = [build_dataset(cfg.data.train)]
    model = build_detector(cfg.model)
    model.init_weights()
    train_detector(model, datasets[0], cfg, distributed=False, validate=True)
    print(f"모델 {model_name} 훈련 완료!")


loading annotations into memory...
Done (t=0.43s)
creating index...
index created!


2024-10-18 02:44:22,421 - mmcv - INFO - initialize ResNet with init_cfg {'type': 'Pretrained', 'checkpoint': 'torchvision://resnet50'}
2024-10-18 02:44:22,422 - mmcv - INFO - load model from: torchvision://resnet50
2024-10-18 02:44:22,423 - mmcv - INFO - load checkpoint from torchvision path: torchvision://resnet50
2024-10-18 02:44:22,531 - mmcv - WARNING - The model and loaded state dict do not match exactly

unexpected key in source state_dict: fc.weight, fc.bias

2024-10-18 02:44:22,546 - mmcv - INFO - initialize FPN with init_cfg {'type': 'Xavier', 'layer': 'Conv2d', 'distribution': 'uniform'}
2024-10-18 02:44:22,587 - mmcv - INFO - initialize RPNHead with init_cfg {'type': 'Normal', 'layer': 'Conv2d', 'std': 0.01}
2024-10-18 02:44:22,604 - mmcv - INFO - initialize Shared2FCBBoxHead with init_cfg [{'type': 'Normal', 'std': 0.01, 'override': {'name': 'fc_cls'}}, {'type': 'Normal', 'std': 0.001, 'override': {'name': 'fc_reg'}}, {'type': 'Xavier', 'distribution': 'uniform', 'override'

loading annotations into memory...
Done (t=0.05s)
creating index...


2024-10-18 02:44:25,039 - mmdet - INFO - Start running, host: root@instance-12525, work_dir: /data/ephemeral/home/KSG-git/level2-objectdetection-cv-20/code/baseline/mmdetection/work_dirs/faster_rcnn_fpn_pos_iou_thr0.9_neg_iou_thr0.5_min_pos_iou0.3_score_thr0.1
2024-10-18 02:44:25,040 - mmdet - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) CheckpointHook                     
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) MyMMDetWandbHook                   
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) StepLrUpdaterHook                  
(NORMAL      ) NumClassCheckHook                  
(LOW         ) IterTimerHook                      
(LOW         ) EvalHook                           
(VERY_LOW    ) TextLoggerHook                     
(VERY_LOW    ) MyMMDetWandbHook                   
 -------------------- 
before_train

index created!


wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


2024-10-18 02:44:31,212 - mmdet - WARNING - No meta information found in the runner. 
wandb: Downloading large artifact val:v1, 152.24MB. 101 files... 
wandb:   101 of 101 files downloaded.  
Done. 0:0:0.8
2024-10-18 02:45:29,304 - mmdet - INFO - Epoch [1][50/2169]	lr: 1.978e-03, eta: 2:18:22, time: 0.192, data_time: 0.047, memory: 1492, loss_rpn_cls: 0.4517, loss_rpn_bbox: 0.0591, loss_cls: 0.4632, acc: 95.5703, loss_bbox: 0.0033, loss: 0.9772, grad_norm: 7.6400
2024-10-18 02:45:34,879 - mmdet - INFO - Epoch [1][100/2169]	lr: 3.976e-03, eta: 1:49:19, time: 0.112, data_time: 0.004, memory: 1492, loss_rpn_cls: 0.1721, loss_rpn_bbox: 0.0556, loss_cls: 0.2038, acc: 97.0645, loss_bbox: 0.0042, loss: 0.4357, grad_norm: 3.9816
2024-10-18 02:45:40,453 - mmdet - INFO - Epoch [1][150/2169]	lr: 5.974e-03, eta: 1:39:34, time: 0.111, data_time: 0.004, memory: 1492, loss_rpn_cls: 0.1462, loss_rpn_bbox: 0.0408, loss_cls: 0.1223, acc: 98.2129, loss_bbox: 0.0039, loss: 0.3133, grad_norm: 2.2521
2024-1

KeyboardInterrupt: 